In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import glob
import os
from os.path import join, basename
from skimage.util import img_as_ubyte
import skimage as ski
from skimage import filters
from skimage import exposure
import pandas as pd

In [2]:
# Your file paths
DATAROOT = r"E:\Documentos\01_DTU\DTU-TFM\Fotos\DOE\1\1 cropped"


image_files = sorted(glob.glob(join(DATAROOT, 'Images', '*.tif')), key=lambda x: int(basename(x).split('.')[0]))

In [3]:
fig, axs = plt.subplots(len(image_files), 2, figsize=(15, 4 * len(image_files)))
t=110
# Create a CLAHE object
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

# Create an empty DataFrame to store density results
density_data = {'Image': [], 'Density': []}

# Create a list to store image labels
image_labels = []

# Calculate the histograms and contrast metric, and populate the subplots
for i in range(len(image_files)):
    read_img = cv2.imread(image_files[i], 0)
    # Apply CLAHE to the image
    img = clahe.apply(read_img)

    # create a mask based on the threshold    
    thresh = (img < t).astype(np.uint8)
    hist = cv2.calcHist([img], [0], None, [256], [0, 256])
    
    # Apply the binary mask to the original image
    result = cv2.bitwise_and(img, img, mask=thresh)

    # Calculate the contrast metric
    rootPixels = np.count_nonzero(thresh)
    w = thresh.shape[1]
    h = thresh.shape[0]
    density = round(rootPixels / (w * h), 3)  # Round to three decimals

    # Add results to the DataFrame
    density_data['Image'].append(basename(image_files[i]))
    density_data['Density'].append(density)


    # Display the image
    plt.subplot(len(image_files), 2, 2 * i + 1)
    plt.imshow(result, cmap='gray')
    plt.title(f'{basename(image_files[i])}')

    # Display the histogram
    plt.subplot(len(image_files), 2, 2 * i + 2)
    plt.hist(result.ravel(), 256, [0, 256])
    plt.title(f'Histogram {i+1}\nPercentage of white: {density}')
    plt.xlabel('Intensity Value')
    plt.ylabel('Count')
    plt.xlim([0, 256])

# Save the DataFrame to a CSV file
density_df = pd.DataFrame(density_data)
density_df.to_csv('density_results_DOE_2-1.csv', index=False)

plt.tight_layout()
plt.show()

ValueError: Number of rows must be a positive integer, not 0

<Figure size 1500x0 with 0 Axes>